In [1568]:
pip install faker

Note: you may need to restart the kernel to use updated packages.


In [1569]:
from cloudant.client import Cloudant
from cloudant.error import CloudantException
from cloudant.result import Result, ResultByKey
from datetime import datetime
from faker import Faker
import random
import json

In [1570]:
username = "b47f2543-95d8-4c01-b864-cf9fcb230ba7-bluemix"
password = "N6i0LnK7GlPfKyf1w2vo_pWYGP254k-g6Y78d32LiDx6"
client = Cloudant.iam(username, password)
client.connect()
databaseName = "maindatabase"
mainDatabase = client.create_database(databaseName)

In [1571]:
geoJsonNyc1 = {"GooglePlaceID": "ChIJs--MqP1YwokRBwAhjXWIHn8", "Coordinates": (40.712772, -74.006058)}
geoJsonNyc2 = {"GooglePlaceID": "ChIJOwg_06VPwokRYv534QaPC8g", "Coordinates": (40.712775, -74.005973)}
geoJsonNyc3 = {"GooglePlaceID": "ChIJ9TRDoBBawokR5gdcflPqvjE", "Coordinates": (40.709504, -74.014671)}
geoJsonNyc4 = {"GooglePlaceID": "ChIJlYpIkBBawokRjyd8kYQpUrc", "Coordinates": (40.709137, -74.013651)}
geoJsonNyc5 = {"GooglePlaceID": "ChIJUXPOvRBawokR2iT8sI7Mq3U", "Coordinates": (40.709137, -74.013651)}
geoJsonNyc6 = {"GooglePlaceID": "ChIJh-L3uxlawokRDwLmkKwLIvY", "Coordinates": (40.711114, -74.010333)}
geoJsonNyc7 = {"GooglePlaceID": "ChIJPTJohhdawokR5-M2cwZ2WDo", "Coordinates": (40.708977, -74.009123)}
geoJsonNyc8 = {"GooglePlaceID": "ChIJ-8HrdxtawokREfw5AkGs_Os", "Coordinates": (40.715036, -74.015840)}
geoJsonNyc9 = {"GooglePlaceID": "ChIJ-ziaVBZawokRj7B7C4qAWPY", "Coordinates": (40.706762, -74.008945)}
geoJsonNyc10 = {"GooglePlaceID": "ChIJNfKP5xdawokRyCFSWap_ogE", "Coordinates": (40.707920, -74.007214)}
geoJsonNyc11 = {"GooglePlaceID": "ChIJ_7B6nRZawokRRg3UBR5XohI", "Coordinates": (40.704642, -74.010322)}
geoJsonNyc12 = {"GooglePlaceID": "ChIJ0fci9hNawokRJVR9hdTAt80", "Coordinates": (40.703950, -74.012354)}
geoJsonNyc13 = {"GooglePlaceID": "ChIJ9TRDoBBawokRJgmlJa_UJ0E", "Coordinates": (40.709797, -74.013888)}
geoJsonNyc14 = {"GooglePlaceID": "ChIJbd_WExZawokRbBhzy1Hc4Ms", "Coordinates": (40.705129, -74.007936)}
geoJsonNyc15 = {"GooglePlaceID": "ChIJuydjIhZawokRG1-vnBMShEw", "Coordinates": (40.705562, -74.007189)}
geoJsonNyc16 = {"GooglePlaceID": "ChIJ7UBKbh9awokRsLNXAfxLoHs", "Coordinates": (40.715947, -74.007367)}
geoJsonNyc17 = {"GooglePlaceID": "ChIJ7xcL8z1awokRPFmLkB9tXHc", "Coordinates": (40.705063, -74.006177)}
geoJsonNyc18 = {"GooglePlaceID": "ChIJ3blcJRRawokRex4mbeb9Ix4", "Coordinates": (40.704129, -74.010336)}
geoJsonNyc19 = {"GooglePlaceID": "ChIJl4BZ5opZwokRUR0n7MrTIEA", "Coordinates": (40.715534, -74.012052)}
geoJsonNyc20 = {"GooglePlaceID": "ChIJodiAGRZawokRHXCBqdWXV54", "Coordinates": (40.706877, -74.011265)}
geoJsonNyc21 = {"GooglePlaceID": "ChIJ3a-_JdJQwokR2SXNohPwSQI", "Coordinates": (40.717755, -74.043143)}


geoJsonList = []
geoJsonList.append(geoJsonNyc1)
geoJsonList.append(geoJsonNyc2)
geoJsonList.append(geoJsonNyc3)
geoJsonList.append(geoJsonNyc4)
geoJsonList.append(geoJsonNyc5)
geoJsonList.append(geoJsonNyc6)
geoJsonList.append(geoJsonNyc7)
geoJsonList.append(geoJsonNyc8)
geoJsonList.append(geoJsonNyc9)
geoJsonList.append(geoJsonNyc10)
geoJsonList.append(geoJsonNyc11)
geoJsonList.append(geoJsonNyc12)
geoJsonList.append(geoJsonNyc13)
geoJsonList.append(geoJsonNyc14)
geoJsonList.append(geoJsonNyc15)
geoJsonList.append(geoJsonNyc16)
geoJsonList.append(geoJsonNyc17)
geoJsonList.append(geoJsonNyc18)
geoJsonList.append(geoJsonNyc19)
geoJsonList.append(geoJsonNyc20)
geoJsonList.append(geoJsonNyc21)
print(geoJsonList)




[{'GooglePlaceID': 'ChIJs--MqP1YwokRBwAhjXWIHn8', 'Coordinates': (40.712772, -74.006058)}, {'GooglePlaceID': 'ChIJOwg_06VPwokRYv534QaPC8g', 'Coordinates': (40.712775, -74.005973)}, {'GooglePlaceID': 'ChIJ9TRDoBBawokR5gdcflPqvjE', 'Coordinates': (40.709504, -74.014671)}, {'GooglePlaceID': 'ChIJlYpIkBBawokRjyd8kYQpUrc', 'Coordinates': (40.709137, -74.013651)}, {'GooglePlaceID': 'ChIJUXPOvRBawokR2iT8sI7Mq3U', 'Coordinates': (40.709137, -74.013651)}, {'GooglePlaceID': 'ChIJh-L3uxlawokRDwLmkKwLIvY', 'Coordinates': (40.711114, -74.010333)}, {'GooglePlaceID': 'ChIJPTJohhdawokR5-M2cwZ2WDo', 'Coordinates': (40.708977, -74.009123)}, {'GooglePlaceID': 'ChIJ-8HrdxtawokREfw5AkGs_Os', 'Coordinates': (40.715036, -74.01584)}, {'GooglePlaceID': 'ChIJ-ziaVBZawokRj7B7C4qAWPY', 'Coordinates': (40.706762, -74.008945)}, {'GooglePlaceID': 'ChIJNfKP5xdawokRyCFSWap_ogE', 'Coordinates': (40.70792, -74.007214)}, {'GooglePlaceID': 'ChIJ_7B6nRZawokRRg3UBR5XohI', 'Coordinates': (40.704642, -74.010322)}, {'GooglePla

In [1572]:
def randomizeCondition() :
    num = random.randint(0, 2)
    if (num == 0) :
        return "Recovered/NoCovid19"
    if (num == 1) :
        return "PotentialCovid19"
    if (num == 2) :
        return "HasCovid19"
status = randomizeCondition()

def randomizeVisitID() :
    
    return random.randint(0, 999999999)

visit_id = randomizeVisitID()
print(visit_id)

customListVisitDictionary = [{str(geoJsonNycHall["Coordinates"]):[randomizeVisitID(), datetime.__str__(datetime(2020, 3, 13)), randomizeCondition()]},
                            {str(geoJsonNycHall["Coordinates"]):[randomizeVisitID(), datetime.__str__(datetime(2020, 3, 13)), randomizeCondition()]},
                            {str(geoJsonNycHall["Coordinates"]):[randomizeVisitID(), datetime.__str__(datetime(2020, 3, 13)), randomizeCondition()]},
                            {str(geoJsonNycHall["Coordinates"]):[randomizeVisitID(), datetime.__str__(datetime(2020, 3, 13)), randomizeCondition()]},
                            {str(geoJsonNycHall["Coordinates"]):[randomizeVisitID(), datetime.__str__(datetime(2020, 3, 13)), randomizeCondition()]},
                            {str(geoJsonNycHall["Coordinates"]):[randomizeVisitID(), datetime.__str__(datetime(2020, 3, 13)), randomizeCondition()]},
                            {str(geoJsonNycHall["Coordinates"]):[randomizeVisitID(), datetime.__str__(datetime(2020, 3, 13)), randomizeCondition()]},
                            {str(geoJsonNycHall["Coordinates"]):[randomizeVisitID(), datetime.__str__(datetime(2020, 3, 13)), randomizeCondition()]},
                            {str(geoJsonNycHall["Coordinates"]):[randomizeVisitID(), datetime.__str__(datetime(2020, 3, 13)), randomizeCondition()]},
                            {str(geoJsonNycHall["Coordinates"]):[randomizeVisitID(), datetime.__str__(datetime(2020, 3, 13)), randomizeCondition()]},
                            {str(geoJsonNycHall["Coordinates"]):[randomizeVisitID(), datetime.__str__(datetime(2020, 3, 13)), randomizeCondition()]},
                            {str(geoJsonNycHall["Coordinates"]):[randomizeVisitID(), datetime.__str__(datetime(2020, 3, 13)), randomizeCondition()]},
                            {str(geoJsonNycHall["Coordinates"]):[randomizeVisitID(), datetime.__str__(datetime(2020, 3, 13)), randomizeCondition()]},
                            {str(geoJsonNycHall["Coordinates"]):[randomizeVisitID(), datetime.__str__(datetime(2020, 3, 13)), randomizeCondition()]},
                            {str(geoJsonNycHall["Coordinates"]):[randomizeVisitID(), datetime.__str__(datetime(2020, 3, 13)), randomizeCondition()]},
                            {str(geoJsonNycHall["Coordinates"]):[randomizeVisitID(), datetime.__str__(datetime(2020, 3, 13)), randomizeCondition()]},
                            {str(geoJsonNycHall["Coordinates"]):[randomizeVisitID(), datetime.__str__(datetime(2020, 3, 13)), randomizeCondition()]},
                            {str(geoJsonNycHall["Coordinates"]):[randomizeVisitID(), datetime.__str__(datetime(2020, 3, 13)), randomizeCondition()]},]


def initListVisitDictionary(geoJsonList) :
    visitListDictionary = []
    for i in range(21):
        visitListDictionary.append({str(geoJsonList[i]["Coordinates"]):[randomizeVisitID(), datetime.__str__(datetime(2020, 3, 13)), randomizeCondition()]})     
    return visitListDictionary

ListVisitDictionary = initListVisitDictionary(geoJsonList)


def randomizeTransactionID() :
    return random.randint(0, 32800000)

def randomizeContributionID() :
    return random.randint(0, 32800000)

def init_ListJsonDocument(ListVisitDictionary, geoJsonList):
    ListJsonDocument = []
    for i in range(len(geoJsonList)):
        jsonDocument = {
            "TransactionID" : randomizeTransactionID(),
            "ContributorID" : randomizeContributionID(),
            "GeoLocation": geoJsonList[i],
            "VisitDictionary": ListVisitDictionary[i]     
        }
        type(jsonDocument)
        ListJsonDocument.append(jsonDocument)
    return ListJsonDocument



686420072


In [1573]:
print(ListVisitDictionary)

[{'(40.712772, -74.006058)': [642405400, '2020-03-13 00:00:00', 'Recovered/NoCovid19']}, {'(40.712775, -74.005973)': [380612960, '2020-03-13 00:00:00', 'Recovered/NoCovid19']}, {'(40.709504, -74.014671)': [966611763, '2020-03-13 00:00:00', 'HasCovid19']}, {'(40.709137, -74.013651)': [849164076, '2020-03-13 00:00:00', 'Recovered/NoCovid19']}, {'(40.709137, -74.013651)': [247750111, '2020-03-13 00:00:00', 'PotentialCovid19']}, {'(40.711114, -74.010333)': [179298274, '2020-03-13 00:00:00', 'HasCovid19']}, {'(40.708977, -74.009123)': [777359634, '2020-03-13 00:00:00', 'HasCovid19']}, {'(40.715036, -74.01584)': [26843641, '2020-03-13 00:00:00', 'Recovered/NoCovid19']}, {'(40.706762, -74.008945)': [104154497, '2020-03-13 00:00:00', 'Recovered/NoCovid19']}, {'(40.70792, -74.007214)': [649770180, '2020-03-13 00:00:00', 'PotentialCovid19']}, {'(40.704642, -74.010322)': [375349515, '2020-03-13 00:00:00', 'PotentialCovid19']}, {'(40.70395, -74.012354)': [606714740, '2020-03-13 00:00:00', 'HasCovi

In [1574]:
ListJsonDocument = init_ListJsonDocument(ListVisitDictionary, geoJsonList)
print(ListJsonDocument)


[{'TransactionID': 25973918, 'ContributorID': 18617271, 'GeoLocation': {'GooglePlaceID': 'ChIJs--MqP1YwokRBwAhjXWIHn8', 'Coordinates': (40.712772, -74.006058)}, 'VisitDictionary': {'(40.712772, -74.006058)': [642405400, '2020-03-13 00:00:00', 'Recovered/NoCovid19']}}, {'TransactionID': 20852966, 'ContributorID': 21567071, 'GeoLocation': {'GooglePlaceID': 'ChIJOwg_06VPwokRYv534QaPC8g', 'Coordinates': (40.712775, -74.005973)}, 'VisitDictionary': {'(40.712775, -74.005973)': [380612960, '2020-03-13 00:00:00', 'Recovered/NoCovid19']}}, {'TransactionID': 20640923, 'ContributorID': 24670737, 'GeoLocation': {'GooglePlaceID': 'ChIJ9TRDoBBawokR5gdcflPqvjE', 'Coordinates': (40.709504, -74.014671)}, 'VisitDictionary': {'(40.709504, -74.014671)': [966611763, '2020-03-13 00:00:00', 'HasCovid19']}}, {'TransactionID': 23330100, 'ContributorID': 22912837, 'GeoLocation': {'GooglePlaceID': 'ChIJlYpIkBBawokRjyd8kYQpUrc', 'Coordinates': (40.709137, -74.013651)}, 'VisitDictionary': {'(40.709137, -74.013651)

In [1575]:
def initListIBMDataBaseDocument(ListJsonDocument) :
    ListIBMDataBaseDocument = []
    for i in ListJsonDocument:
        newDocument = mainDatabase.create_document(i)
        ListIBMDataBaseDocument.append(newDocument)
    return ListIBMDataBaseDocument
ListIBMDataBaseDocument = initListIBMDataBaseDocument(ListJsonDocument)

In [1576]:
print(ListIBMDataBaseDocument)

[{'TransactionID': 25973918, 'ContributorID': 18617271, 'GeoLocation': {'GooglePlaceID': 'ChIJs--MqP1YwokRBwAhjXWIHn8', 'Coordinates': (40.712772, -74.006058)}, 'VisitDictionary': {'(40.712772, -74.006058)': [642405400, '2020-03-13 00:00:00', 'Recovered/NoCovid19']}, '_id': 'ea26d825def665b3b5749bd7f7c3216e', '_rev': '1-988c6ca689aceb876b5bf8b82035bae0'}, {'TransactionID': 20852966, 'ContributorID': 21567071, 'GeoLocation': {'GooglePlaceID': 'ChIJOwg_06VPwokRYv534QaPC8g', 'Coordinates': (40.712775, -74.005973)}, 'VisitDictionary': {'(40.712775, -74.005973)': [380612960, '2020-03-13 00:00:00', 'Recovered/NoCovid19']}, '_id': '04af3388a9fba22213feda785cec76d7', '_rev': '1-db593cd44020cff802c5b57c678624cd'}, {'TransactionID': 20640923, 'ContributorID': 24670737, 'GeoLocation': {'GooglePlaceID': 'ChIJ9TRDoBBawokR5gdcflPqvjE', 'Coordinates': (40.709504, -74.014671)}, 'VisitDictionary': {'(40.709504, -74.014671)': [966611763, '2020-03-13 00:00:00', 'HasCovid19']}, '_id': 'f6eaae051748a00c8b4

In [1577]:
def severityOfArea(ListVisitDictionary, geoJsonObject) :
    locationSeverityDict = {}
    highCount = 0
    mediumCount = 0
    lowCount = 0
    highWeight = 1
    mediumWeight = 0.5
    lowWeight = 0
    weightedSum = 0
    for subDict in ListVisitDictionary:
        print(subDict)
        for key in subDict:
            strGeoJsonCoordinates = str(geoJsonObject["Coordinates"])
            if (key == strGeoJsonCoordinates):
                if (subDict[key][2].lower() == "HasCovid19".lower()):
                    highCount = highCount + 1
                elif (subDict[key][2].lower() == "PotentialCovid19".lower()):
                    mediumCount = mediumCount + 1
                else:
                    lowCount = lowCount + 1
    weightedSum = ((1 * highCount) + (0.5 * mediumCount) + (lowCount * lowWeight ))
    locationSeverityDict[geoJsonObject["Coordinates"]] = weightedSum
    return locationSeverityDict

LocationSeverityDict = severityOfArea(ListVisitDictionary, geoJsonNyc21)
print(LocationSeverityDict)

{'(40.712772, -74.006058)': [642405400, '2020-03-13 00:00:00', 'Recovered/NoCovid19']}
{'(40.712775, -74.005973)': [380612960, '2020-03-13 00:00:00', 'Recovered/NoCovid19']}
{'(40.709504, -74.014671)': [966611763, '2020-03-13 00:00:00', 'HasCovid19']}
{'(40.709137, -74.013651)': [849164076, '2020-03-13 00:00:00', 'Recovered/NoCovid19']}
{'(40.709137, -74.013651)': [247750111, '2020-03-13 00:00:00', 'PotentialCovid19']}
{'(40.711114, -74.010333)': [179298274, '2020-03-13 00:00:00', 'HasCovid19']}
{'(40.708977, -74.009123)': [777359634, '2020-03-13 00:00:00', 'HasCovid19']}
{'(40.715036, -74.01584)': [26843641, '2020-03-13 00:00:00', 'Recovered/NoCovid19']}
{'(40.706762, -74.008945)': [104154497, '2020-03-13 00:00:00', 'Recovered/NoCovid19']}
{'(40.70792, -74.007214)': [649770180, '2020-03-13 00:00:00', 'PotentialCovid19']}
{'(40.704642, -74.010322)': [375349515, '2020-03-13 00:00:00', 'PotentialCovid19']}
{'(40.70395, -74.012354)': [606714740, '2020-03-13 00:00:00', 'HasCovid19']}
{'(40